In [1]:
import cv2
import numpy as np
import os
from utils import *
import matplotlib.pyplot as plt


### 🧮 Filtro de paso alto utilizando suavizado con Mediana

La función `filtro_paso_alto_mediana` aplica un filtro de paso alto a una imagen en escala de grises utilizando un suavizado con filtro **mediana**.

**¿Cómo funciona?**
- Primero suaviza la imagen usando un filtro de mediana, que reemplaza cada píxel por el valor mediano de sus vecinos.
- Luego, se resta esta versión suavizada de la imagen original. Esto elimina las bajas frecuencias (zonas suaves) y conserva las altas (bordes y cambios bruscos).

**Ventaja del filtro mediana:**
- Es muy eficaz para eliminar ruido impulsivo (como el ruido "sal y pimienta").
- A diferencia del filtro de la media o gaussiano, **preserva mejor los bordes**, lo cual lo hace útil cuando se necesita resaltar detalles sin distorsionar contornos importantes.

Este filtro es ideal cuando se quiere detectar detalles sin verse tan afectado por el ruido.

In [2]:
def filtro_paso_alto_mediana(img_gray, ksize=5):
    suavizada = cv2.medianBlur(img_gray, ksize)
    return cv2.subtract(img_gray, suavizada)


### 🗂️ Procesamiento masivo de imágenes con filtro de la Mediana

La función `procesar_imagenes_carpeta_mediana` aplica un filtro de paso alto basado en el suavizado por mediana a todas las imágenes `.png` contenidas en una estructura de carpetas.

**Qué hace:**
- Recorre todas las subcarpetas de una carpeta principal.
- Aplica el filtro de paso alto por mediana a cada imagen en escala de grises.
- Guarda las imágenes procesadas respetando la estructura de carpetas.
- Calcula la varianza de alta frecuencia de cada imagen y la guarda en un archivo `.txt`.

**¿Por qué usar el filtro de mediana?**
El filtro de mediana suaviza la imagen eliminando ruido tipo "sal y pimienta" sin distorsionar los bordes, lo que lo hace especialmente útil cuando se necesita conservar detalles importantes. Al restar esta imagen suavizada de la original, se obtienen las altas frecuencias de forma robusta.

**Utilidad:**
Este procesamiento es ideal para análisis de bordes y detección de detalles en presencia de ruido, facilitando comparaciones con otros tipos de filtrado como el gaussiano o el de la media.


In [3]:
def procesar_imagenes_carpeta_mediana(
    carpeta_entrada="images",
    carpeta_salida="images_procesadas",
    ksize=5
):
    os.makedirs(carpeta_salida, exist_ok=True)

    for subcarpeta in sorted(os.listdir(carpeta_entrada)):
        ruta_subcarpeta = os.path.join(carpeta_entrada, subcarpeta)
        if not os.path.isdir(ruta_subcarpeta):
            continue

        carpeta_salida_sub = os.path.join(carpeta_salida, subcarpeta, "mediana")
        os.makedirs(carpeta_salida_sub, exist_ok=True)

        ruta_txt = os.path.join(carpeta_salida_sub, "info_varianzas_mediana.txt")
        with open(ruta_txt, "w", encoding="utf-8") as archivo_txt:
            archivo_txt.write("Varianzas de alta frecuencia (filtro de la mediana)\n")
            archivo_txt.write(f"Carpeta de imágenes: {ruta_subcarpeta}\n\n")
            archivo_txt.write("La varianza indica la dispersión de los valores de alta frecuencia.\n")
            archivo_txt.write("Valores mayores suelen corresponder a más bordes y mayor nitidez.\n\n")

            for filename in sorted(os.listdir(ruta_subcarpeta)):
                if not filename.lower().endswith(".png"):
                    continue

                ruta_imagen_entrada = os.path.join(ruta_subcarpeta, filename)
                img_bgr = cv2.imread(ruta_imagen_entrada)
                if img_bgr is None:
                    continue

                img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

                # Aplicar filtro y calcular varianza
                altas_frec_img = filtro_paso_alto_mediana(img_gray, ksize=ksize)
                varianza = np.var(altas_frec_img)

                # Normalización para mejorar contraste
                hf_min, hf_max = altas_frec_img.min(), altas_frec_img.max()
                if hf_max > hf_min:
                    altas_frec_escalada = ((altas_frec_img - hf_min) / (hf_max - hf_min) * 255).astype(np.uint8)
                else:
                    altas_frec_escalada = altas_frec_img.astype(np.uint8)

                nombre, _ = os.path.splitext(filename)
                ruta_imagen_salida = os.path.join(carpeta_salida_sub, f"{nombre}_alta_frecuencia_mediana.png")

                cv2.imwrite(ruta_imagen_salida, altas_frec_escalada)
                archivo_txt.write(f"{filename} -> varianza: {varianza:.3f}\n")

    print("Procesamiento completado con filtro de la Mediana.")


In [4]:
# Ejecuta el procesamiento
procesar_imagenes_carpeta_mediana(
    carpeta_entrada="images",
    carpeta_salida="images_procesadas",
    ksize=5
)


Procesamiento completado con filtro de la Mediana.
